# CEWA 568 Snow Hydrology
## Homework #5: *Radiation, Albedo, and the Energy Balance* 
#### Author: Cassie Lumbrazo

In [ ]:
#first import the relevant python packages
import xarray as xr
import numpy as np
import os 
import urllib
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

import json
import pytz

import seaborn as sns 

sns.set_theme()
plt.rcParams['figure.figsize'] = [12,6] #overriding size

In [ ]:
# We want to install a package called pysolar, and we need pip for the JupyterHub

#from pysolar.solar import *
!pip install pysolar

In [ ]:
# Now that we've installed that package, we need to import all of its functions
from pysolar.solar import *

## Get set up from Lab 5-1. 

*Code from lab 5-1. sos-dataset*

In [ ]:
sos_ds=xr.open_dataset("sos_ds.nc")
sos_ds.head()

*Code from Lab5-1. radsys*

In [ ]:
radsys_ds=xr.open_dataset("radsys_ds.nc")
radsys_ds.head()

*pysolar for potential radiation based on location, but not local topography*

In [ ]:
import pytz
kettle_ponds_lat_lon = [-106.97298,38.94182]
dates = pd.date_range(dt.datetime(2022,12,28,0,0),dt.datetime(2023,5,10,0,0),  freq='1H') # changed the dates here from the lab

clear_sky_rad = []
for date in dates:
    date = (pytz.utc.localize(date)).to_pydatetime()
    altitude_deg = get_altitude(kettle_ponds_lat_lon[1], kettle_ponds_lat_lon[0], date)
    clear_sky_rad.append(radiation.get_radiation_direct(date, altitude_deg))

In [ ]:
plt.plot(dates,clear_sky_rad)

plt.title('pysolar for the entire winter at Kettle Ponds')
plt.ylabel("Radiation [W/m^2]")

In [ ]:
plt.plot(dates,clear_sky_rad)

plt.title('pysolar for just the period in January we will focus on')
plt.ylabel("Radiation [W/m^2]")
plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,10))

# Begin Homework # 5

In [ ]:
!pwd

In [ ]:
precip=xr.open_dataset("/home/jovyan/Week2/precipitation.nc")
precip.head()

In [ ]:
# code from Danny to make accum precip, daily precip
# First we will plot the precipitation time series to look at when the upward pointing radiometers may be comprimized
daily_precip = precip['acc_prec'].diff(dim='date').to_series()
daily_precip.index = precip['acc_prec'].diff(dim='date').to_series().index.date

In [ ]:
type(daily_precip)

In [ ]:
precip['acc_prec'].plot(label='accumulated precip')
daily_precip.plot(label='calculated daily value', marker='o', markersize=10, color='blue')

plt.legend()
plt.title('precip')

In [ ]:
precip['acc_prec'].plot(label='accumulated precip')
daily_precip.plot(label='calculated daily value', marker='o', markersize=10, color='blue')
plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,15))

plt.legend()
plt.title('precip')

## Problem 1: Comparing solar radiation sensors
**A common problem in mountain snow energy balance studies is that snow accumulates on the upward pointing radiometers. Find a time in our dataset when you think this occurred and explain your reasoning. Hint, you may want to look at the precipitation dataset in Lab 2 for timing. Which radiometer set-up (SOS or SPLASH) worked better during your timeperiod? Why do you think this is? Compare downwelling and reflected shortwave radiation with potential shortwave radiation for your day.**

In [ ]:
# putting the clear sky radiation into a dataframe so that I can plot it with everything else more easily
clear_rad = pd.DataFrame()
clear_rad['dates'] = dates
clear_rad['clear_sky_rad'] = clear_sky_rad
clear_rad.set_index('dates', inplace=True)

clear_df = clear_rad.to_xarray()
clear_df

### Looking at Radsys Data
* need to find what data to plot
* and what data is clean (or need to clean it up a little to look)

In [ ]:
# need to add radsys_ds data 
radsys_ds.head()

In [ ]:
radsys_ds.dw_solar.plot(label='dw solar')
radsys_ds.dw_solar_qc.plot(label='dw solar qc')
radsys_ds.uw_solar.plot(label='uw solar')
radsys_ds.uw_solar_qc.plot(label='uw solar qc')
radsys_ds.NetSolar.plot(label='net solar')
radsys_ds.NetSolar_qc.plot(label='net solar qc')

plt.title('radsys data')
plt.ylabel("Radiation [W/m^2]")
plt.legend()

In [ ]:
radsys_ds.uw_ir.plot(label='dw ir')
radsys_ds.uw_ir_qc.plot(label='dw ir qc')

radsys_ds.dw_ir.plot(label='uw ir')
radsys_ds.dw_ir_qc.plot(label='uw ir qc')

plt.title('radsys data')
plt.ylabel("Radiation [W/m^2]")
plt.legend()

### Removing negatives from the radsys dataset
Even the "QC" data has these values? I think they are -9999 for no data. 
* making those nan to make plotting easier

In [ ]:
# removing all the weird negative values just so I can look
clean_dw_solar = radsys_ds.dw_solar.where(radsys_ds.dw_solar > -10)
clean_uw_solar = radsys_ds.uw_solar.where(radsys_ds.uw_solar > -10)
clean_net_solar = radsys_ds.NetSolar.where(radsys_ds.NetSolar > -10)
clean_dw_ir = radsys_ds.dw_ir.where(radsys_ds.dw_ir > -10)
clean_uw_ir = radsys_ds.uw_ir.where(radsys_ds.uw_ir > -10)

In [ ]:
clean_dw_solar.plot( label='dw solar  (removed negatives)', linewidth=2, color='gold')
clean_uw_solar.plot( label='uw solar  (removed negatives)', linewidth=2, color='orange')
clean_net_solar.plot(label='net solar (removed negatives)', linewidth=2, color='grey')

clean_dw_ir.plot(label='dw ir (removed negatives)', linewidth=2, color='darkblue')
clean_uw_ir.plot(label='uw ir (removed negatives)', linewidth=2, color='blue')

plt.title('radsys data')
plt.ylabel("Radiation [W/m^2]")
plt.legend()

*I'm guessign the names,* 
* **dw ir** means, **d**own**w**welling IR 
* **uw ir** means, **u**p**w**welling IR 

### Looking at sos data 

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15,8), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 1]})

# sos data plot
sos_ds['Rsw_in_9m_d'].plot(ax=axes[0], label='downward shortwave', linewidth=3, color='gold')
sos_ds['Rsw_out_9m_d'].plot(ax=axes[0], label='upward shortwave',linewidth=3, color='orange')
sos_ds['LWin'].plot(ax=axes[0], label='downward longwave', linewidth=3, color='royalblue')
sos_ds['LWout'].plot(ax=axes[0], label='upward longwave', linewidth=3, color='mediumblue')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')

# daily precip
daily_precip.plot(ax=axes[1], marker='o', markersize=10, label='precipitation', color='dodgerblue')


# legends et al. 
axes[0].set_title('sos data')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].legend(loc='upper right')
axes[1].set_ylabel("Daily Precipitation")

plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,9))
plt.show()

## Put all the datasets together
* sos
* radsys
* pysolar
* precip

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15,10), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 2, 1]})

# sos data plot
sos_ds['Rsw_in_9m_d'].plot(ax=axes[0], label='sosdat: downward shortwave', linewidth=3, color='gold')
sos_ds['Rsw_out_9m_d'].plot(ax=axes[0], label='sosdat: upward shortwave',linewidth=3, color='darkgoldenrod')
sos_ds['LWin'].plot(ax=axes[0], label='sosdat: downward longwave', linewidth=3, color='cornflowerblue')
sos_ds['LWout'].plot(ax=axes[0], label='sosdat: upward longwave', linewidth=3, color='mediumblue')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')


# radsys data plot
clean_dw_solar.plot(ax=axes[1], label='radsys: downward shortwave', linewidth=3, color='gold')
clean_uw_solar.plot(ax=axes[1], label='radsys: upward shortwave', linewidth=3, color='darkgoldenrod')
clean_net_solar.plot(ax=axes[1],label='radsys: net solar', linewidth=2, color='black')

clean_dw_ir.plot(ax=axes[1],label='radsys: downward longwave', linewidth=3, color='cornflowerblue')
clean_uw_ir.plot(ax=axes[1],label='radsys: upward longwave', linewidth=3, color='mediumblue')

clear_rad.plot(ax=axes[1], label='pysolar: clear sky rad', linewidth=1, color='grey')


# daily precip
daily_precip.plot(ax=axes[2], marker='o', markersize=10, label='precipitation', color='deepskyblue')


# legends et al. 
axes[0].set_title('sos data')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].set_title('radsys data')
axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_ylabel("Radiation [W/m^2]")

axes[2].legend(loc='upper right')
axes[2].set_ylabel("Daily Precipitation")

plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,9))
plt.show()

### plotting with all shortwave and all longwave on the same plots to better compare sos to radsys

In [ ]:
# plotting all shortwave and longwaves together (not by dataset)

fig, axes = plt.subplots(3, 1, figsize=(17,12), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 2, 1]})

# shortwave plot
sos_ds['Rsw_in_9m_d'].plot(ax=axes[0], label='sosdat: downward shortwave', linewidth=3, color='gold')
sos_ds['Rsw_out_9m_d'].plot(ax=axes[0], label='sosdat: upward shortwave',linewidth=3, color='darkgoldenrod')

clean_dw_solar.plot(ax=axes[0], label='radsys: downward shortwave', linewidth=3, color='tomato')
clean_uw_solar.plot(ax=axes[0], label='radsys: upward shortwave', linewidth=3, color='darkred')
clean_net_solar.plot(ax=axes[0],label='radsys: net solar', linewidth=2, color='black')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')


# longwave plot
sos_ds['LWin'].plot(ax=axes[1], label='sosdat: downward longwave', linewidth=3, color='cornflowerblue')
sos_ds['LWout'].plot(ax=axes[1], label='sosdat: upward longwave', linewidth=3, color='mediumblue')

clean_dw_ir.plot(ax=axes[1],label='radsys: downward longwave', linewidth=3, color='darkcyan')
clean_uw_ir.plot(ax=axes[1],label='radsys: upward longwave', linewidth=3, color='darkslategrey')


# daily precip
daily_precip.plot(ax=axes[2], marker='o', markersize=10, label='precipitation', color='deepskyblue')


# legends et al. 
axes[0].set_title('shortwave data')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].set_title('longwave data')
axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_ylabel("Radiation [W/m^2]")

axes[2].legend(loc='upper right')
axes[2].set_ylabel("Daily Precipitation")

plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,9))
plt.show()

### highlighting then there is snow on the sensors

In [ ]:
# plotting all shortwave and longwaves together (not by dataset)

fig, axes = plt.subplots(3, 1, figsize=(17,12), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 2, 1]})

# shortwave plot
sos_ds['Rsw_in_9m_d'].plot(ax=axes[0], label='sosdat: downward shortwave', linewidth=3, color='gold')
sos_ds['Rsw_out_9m_d'].plot(ax=axes[0], label='sosdat: upward shortwave',linewidth=3, color='darkgoldenrod')

clean_dw_solar.plot(ax=axes[0], label='radsys: downward shortwave', linewidth=3, color='tomato')
clean_uw_solar.plot(ax=axes[0], label='radsys: upward shortwave', linewidth=3, color='darkred')
clean_net_solar.plot(ax=axes[0],label='radsys: net solar', linewidth=2, color='black')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')


# longwave plot
sos_ds['LWin'].plot(ax=axes[1], label='sosdat: downward longwave', linewidth=3, color='cornflowerblue')
sos_ds['LWout'].plot(ax=axes[1], label='sosdat: upward longwave', linewidth=3, color='mediumblue')

clean_dw_ir.plot(ax=axes[1],label='radsys: downward longwave', linewidth=3, color='darkcyan')
clean_uw_ir.plot(ax=axes[1],label='radsys: upward longwave', linewidth=3, color='darkslategrey')


# daily precip
daily_precip.plot(ax=axes[2], marker='o', markersize=10, label='precipitation', color='deepskyblue')


# shade on graph for precipitation event and then snow on the sensor 
axes[0].axvspan(dt.datetime(2022,12,31), dt.datetime(2023,1,4), alpha=0.3, color='lightblue')
axes[0].axvspan(dt.datetime(2023,1,6,12), dt.datetime(2023,1,7), alpha=0.3, color='lightblue')


# legends et al. 
axes[0].set_title('shortwave data')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].set_title('longwave data')
axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_ylabel("Radiation [W/m^2]")

axes[2].legend(loc='upper right')
axes[2].set_ylabel("Daily Precipitation")

plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,9))
plt.show()

### zooming into the region when there is snow on the sensors

In [ ]:
# plotting all shortwave and longwaves together (not by dataset)

fig, axes = plt.subplots(3, 1, figsize=(17,12), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 2, 1]})

# shortwave plot
sos_ds['Rsw_in_9m_d'].plot(ax=axes[0], label='sosdat: downward shortwave', linewidth=3, color='gold')
sos_ds['Rsw_out_9m_d'].plot(ax=axes[0], label='sosdat: upward shortwave',linewidth=3, color='darkgoldenrod')

clean_dw_solar.plot(ax=axes[0], label='radsys: downward shortwave', linewidth=3, color='tomato')
clean_uw_solar.plot(ax=axes[0], label='radsys: upward shortwave', linewidth=3, color='darkred')
clean_net_solar.plot(ax=axes[0],label='radsys: net solar', linewidth=2, color='black')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')


# longwave plot
sos_ds['LWin'].plot(ax=axes[1], label='sosdat: downward longwave', linewidth=3, color='cornflowerblue')
sos_ds['LWout'].plot(ax=axes[1], label='sosdat: upward longwave', linewidth=3, color='mediumblue')

clean_dw_ir.plot(ax=axes[1],label='radsys: downward longwave', linewidth=3, color='darkcyan')
clean_uw_ir.plot(ax=axes[1],label='radsys: upward longwave', linewidth=3, color='darkslategrey')


# daily precip
daily_precip.plot(ax=axes[2], marker='o', markersize=10, label='precipitation', color='deepskyblue')


# shade on graph for precipitation event and then snow on the sensor 
axes[0].axvspan(dt.datetime(2022,12,31), dt.datetime(2023,1,4), alpha=0.3, color='lightblue')
axes[0].axvspan(dt.datetime(2023,1,6,12), dt.datetime(2023,1,7), alpha=0.3, color='lightblue')

axes[1].axvspan(dt.datetime(2022,12,31), dt.datetime(2023,1,4), alpha=0.3, color='lightblue')
axes[1].axvspan(dt.datetime(2023,1,6,12), dt.datetime(2023,1,7), alpha=0.3, color='lightblue')

axes[2].axvspan(dt.datetime(2022,12,31), dt.datetime(2023,1,4), alpha=0.3, color='lightblue')
axes[2].axvspan(dt.datetime(2023,1,6,12), dt.datetime(2023,1,7), alpha=0.3, color='lightblue')


# legends et al. 
axes[0].set_title('shortwave data')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].set_title('longwave data')
axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_ylabel("Radiation [W/m^2]")

axes[2].legend(loc='upper right')
axes[2].set_ylabel("Daily Precipitation")

plt.xlim(dt.datetime(2023,1,5), dt.datetime(2023,1,8))
plt.show()

___________________________
## Problem 1: Solution
* When snow accumulates on upward pointing radiometers, the values for the upward pointing radiometer will decrease significantly. 
* This obviously is only going to happen when there is snowfall (precip) and then the values for the radiometer will stay that way until someone goes and cleans the snow off (or it melts) because it has a nice fancy heating system.  

Compare downwelling and reflected shortwave radiation with potential shortwave radiation for that day. 
* When the data shows that there is more downwelling radiation than upwelling radiation, then we have snow on the sensors. There can not physically be more upwelling shortwave radiation than there is incoming shortwave radiation
* There are quite a few times during this short period when there is snow on the SOS downwelling shortwave radiation sensor (highlighted in blue on the above plots)
* It looks like the SPLASH (radsys) radiometers worked better for this time period. I think they are heated? 
___________________________

## Problem 2: Clouds
**Identify a period of variable cloud cover in the dataset. Explain how you can use both shortwave and longwave measurements to identify variations in clouds. include periods from both day and nighttime hours. How are the shortwave and longwave datasets complimentary? Do they tell you the same or different information about the clouds?**

In [ ]:
# plotting all shortwave and longwaves together (not by dataset)

fig, axes = plt.subplots(3, 1, figsize=(17,12), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 2, 1]})

# shortwave plot
sos_ds['Rsw_in_9m_d'].plot(ax=axes[0], label='sosdat: downward shortwave', linewidth=3, color='gold')
sos_ds['Rsw_out_9m_d'].plot(ax=axes[0], label='sosdat: upward shortwave',linewidth=3, color='darkgoldenrod')

clean_dw_solar.plot(ax=axes[0], label='radsys: downward shortwave', linewidth=3, color='tomato')
clean_uw_solar.plot(ax=axes[0], label='radsys: upward shortwave', linewidth=3, color='darkred')
clean_net_solar.plot(ax=axes[0],label='radsys: net solar', linewidth=2, color='black')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')


# longwave plot
sos_ds['LWin'].plot(ax=axes[1], label='sosdat: downward longwave', linewidth=3, color='cornflowerblue')
sos_ds['LWout'].plot(ax=axes[1], label='sosdat: upward longwave', linewidth=3, color='mediumblue')

clean_dw_ir.plot(ax=axes[1],label='radsys: downward longwave', linewidth=3, color='darkcyan')
clean_uw_ir.plot(ax=axes[1],label='radsys: upward longwave', linewidth=3, color='darkslategrey')


# daily precip
daily_precip.plot(ax=axes[2], marker='o', markersize=10, label='precipitation', color='deepskyblue')


# shade on graph for period of cloud cover
axes[0].axvspan(dt.datetime(2022,12,31,12), dt.datetime(2023,1,2,2), alpha=0.3, color='lightblue')

axes[1].axvspan(dt.datetime(2022,12,31,12), dt.datetime(2023,1,2,2), alpha=0.3, color='lightblue')

axes[2].axvspan(dt.datetime(2022,12,31,12), dt.datetime(2023,1,2,2), alpha=0.3, color='lightblue')



# legends et al. 
axes[0].set_title('shortwave data')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].set_title('longwave data')
axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_ylabel("Radiation [W/m^2]")

axes[2].legend(loc='upper right')
axes[2].set_ylabel("Daily Precipitation")

plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,9))
plt.show()

__________________________
## Problem 2: Solution
*We want to use downwelling shortwave radiation and downwelling longwave radiation to see clouds.* 
* Downwelling (incoming) longwave is large when there are clouds (i.e., clouds emit longwave radiation). This occurs day and night. 
* Downwelling (incoming) shortwave radiation is small when there are clouds (i.e., sun is blocked by clouds). This occurs during the day. 

They tell you different information about clouds. Downwelling longwave will tell you how much radiation the clouds are emitting, while downwelling shortwave can tell you how much shortwave is either A) directly making it through the clouds or B) being scattered by the clouds. I think Jessica mentioned in class that someone has an instrument at Kettle Ponds that is measuring scattered incoming shortwave vs direct incoming shortwave radiation. This would be a good way to get more information about the clouds as well (at least, during the day). 

__________________________

## Problem 3: Dust on snow and albedo
**We know that the reflectivity of snow, termed albedo, calculated as outgoing-solar-radiation divided by incoming-solar-radiation, is brightest right after new snowfall and then darkens as snow ages. This occurs both as a process of snow grains rounding and growing and as snow gets dirtier with deposition. In early April, a substantial amount of dust was deposited on our site at Kettle Ponds (see the photo at the top of this page). Using the Kettle Ponds radiation dataset, investigate how albedo changes both with time after a new snowfall event earlier in the winter (no dust) and in mid-April (with dust). How much does dust impact albedo compared to the natural snow aging process?**

In [ ]:
# create a new dataframe in lab05-1 for the april time period and open it here to save myself all the code inside this notebook to do that 
sos_ds_april=xr.open_dataset("sos_ds_april.nc")
sos_ds_april

In [ ]:
# calculate outgoing solar radiation by dividing by incoming solar radiation 
sos_ds['albedo'] = sos_ds['Rsw_out_9m_d'] / sos_ds['Rsw_in_9m_d']
sos_ds_april['albedo'] = sos_ds_april['Rsw_out_9m_d'] / sos_ds_april['Rsw_in_9m_d']

In [ ]:
# # clean the outliers and weird values so we can see on a range from 0 to 1 
# sos_ds['albedo'] = sos_ds['albedo'].where(sos_ds['albedo'] >= 0)
# sos_ds_april['albedo'] = sos_ds_april['albedo'].where(sos_ds_april['albedo'] >= -1)

# sos_ds['albedo'] = sos_ds['albedo'].where(sos_ds['albedo'] <= 1)
# sos_ds_april['albedo'] = sos_ds_april['albedo'].where(sos_ds_april['albedo'] <= 3)

In [ ]:
# plotting all shortwave and longwaves together (not by dataset)

fig, axes = plt.subplots(3, 1, figsize=(15,10), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 2, 1]})

# shortwave plot
sos_ds['Rsw_in_9m_d'].plot(ax=axes[0], label='sos: downward shortwave', linewidth=3, color='gold')
sos_ds['Rsw_out_9m_d'].plot(ax=axes[0], label='sos: upward shortwave',linewidth=3, color='darkgoldenrod')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')

# albedo
sos_ds['albedo'].plot(ax=axes[1], label='sos: calculated albedo',  marker='o', markersize=5,  color='black', linestyle='')
axes[1].set_ylim([0,1])

# daily precip
daily_precip.plot(ax=axes[2], marker='o', markersize=10, label='precipitation', color='dodgerblue')


# legends et al. 
axes[0].set_title('January: shortwave data "without dust"')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].set_title('January: albedo "without dust"')
axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_ylabel("calculated albedo \n downward shortwave / upward shortwave")

axes[2].legend(loc='upper right')
axes[2].set_ylabel("Daily Precipitation")


plt.xlim(dt.datetime(2022,12,30), dt.datetime(2023,1,9))
plt.show()

### going to try to look at early march into mid-april to compare albedo before and after the dust directly 

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15,10), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [2, 2, 2]})

# shortwave plot
sos_ds_april['Rsw_in_9m_d'].plot(ax=axes[0], label='sos: downward shortwave', linewidth=3, color='gold')
sos_ds_april['Rsw_out_9m_d'].plot(ax=axes[0], label='sos: upward shortwave',linewidth=3, color='darkgoldenrod')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')


# longwave plot
sos_ds_april['LWin'].plot(ax=axes[1], label='sos: downward longwave (in)', linewidth=3, color='royalblue')
sos_ds_april['LWout'].plot(ax=axes[1], label='sos: upward longwave (out)', linewidth=3, color='mediumblue')

# albedo
sos_ds_april['albedo'].plot(ax=axes[2], label='sos: calculated albedo',  marker='o', markersize=5,  color='black', linestyle='')
axes[2].set_ylim([0,1])

# # daily precip
# daily_precip.plot(ax=axes[3], marker='o', markersize=10, label='precipitation', color='dodgerblue') # precip data doesn't go until April?

# shade on graph for precipitation event and then snow on the sensor 
axes[0].axvspan(dt.datetime(2023,4,4, 12), dt.datetime(2023,4,14), alpha=0.2, color='grey')
axes[1].axvspan(dt.datetime(2023,4,4, 12), dt.datetime(2023,4,14), alpha=0.2, color='grey')
axes[2].axvspan(dt.datetime(2023,4,4, 12), dt.datetime(2023,4,14), alpha=0.2, color='grey')

# legends et al. 
axes[0].set_title('April: shortwave data without dust vs with dust')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].set_title('April: longwave data without dust vs with dust')
axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_ylabel("Radiation [W/m^2]")

axes[2].set_title('April: albedo without dust vs with dust')
axes[2].legend(loc='upper right')
axes[2].set_xlabel("")
axes[2].set_ylabel("calculated albedo \n downward shortwave / upward shortwave")

# axes[3].legend(loc='upper right')
# axes[3].set_ylabel("Daily Precipitation")

plt.xlim(dt.datetime(2023,3,27), dt.datetime(2023,4,19))
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15,10), sharex=True, constrained_layout = True, gridspec_kw={'height_ratios': [1, 3]})

# shortwave plot
sos_ds_april['Rsw_in_9m_d'].plot(ax=axes[0], label='sos: downward shortwave', linewidth=3, color='gold')
sos_ds_april['Rsw_out_9m_d'].plot(ax=axes[0], label='sos: upward shortwave',linewidth=3, color='darkgoldenrod')

clear_rad.plot(ax=axes[0], label='clear sky rad', linewidth=1, color='grey')

# albedo
sos_ds_april['albedo'].plot(ax=axes[1], label='sos: calculated albedo', marker='o', markersize=5,  color='black', linestyle='')


# shade on graph for precipitation event and then snow on the sensor 
axes[0].axvspan(dt.datetime(2023,4,4, 12), dt.datetime(2023,4,14), alpha=0.2, color='grey')
axes[1].axvspan(dt.datetime(2023,4,4, 12), dt.datetime(2023,4,14), alpha=0.2, color='grey')

# legends et al. 
axes[0].set_title('April: shortwave data without dust vs with dust')
axes[0].legend(loc='upper right')
axes[0].set_xlabel("")
axes[0].set_ylabel("Radiation [W/m^2]")

axes[1].legend(loc='upper right')
axes[1].set_xlabel("")
axes[1].set_title('April: albedo without dust vs with dust')
axes[1].set_ylabel("calculated albedo \n downward shortwave / upward shortwave")

axes[1].set_ylim([0,1])

axes[0].set_xlim(dt.datetime(2023,3,27), dt.datetime(2023,4,13,12))
axes[1].set_xlim(dt.datetime(2023,3,27), dt.datetime(2023,4,13,12))
plt.show()

__________________________
## Problem 3: Solution
* Albedo is brightest right after new snowfall, and then darkens as the snow ages 
* This is a positive feedback loop when the snow becomes dirty -> albedo decreases -> snow grains increase in size more quickly -> albedo decreases even more. 
* In early April, a lot of dust was depositied at the Kettle Ponds site. We calculate albedo using outgoing solar radiation / incoming dolar radiation, and see how it changes after the dust storm in April
* Just a note: most of the time in April the SOS downwelling shortwave radiation goes over the pysolar radiation. I had this similar thing when using pysolar for my research, it considers lat,lon but it does not consider elevation or topography in the calculation of potential radiation. So perhaps this is the reason? or is there another reason this would happen? 

**The albedo in late March (without dust) and early-mid April (with dust; *highlighted in grey on plots*)**
* The grey shading indicates when there is dust on the snow after the early april event. 
* I limited the calculated albedo between 0 and 1 since there are some strange values from uncleaned data. 
* There is certainly a general trend of decreasing albedo during this highlighted period. The general values go from ranging of 1-0.8 to between 0.8 and 0.6. 
* As we discussed in our energy balance lecture in class, dirty snow impacts the albedo of the snow surface a lot. I cannot remember the numbers right now from the slides (and I can't find the slides) but I think the numbers were ~30% decrease in the albedo compared to fresh snow from the conceptual diagram? A similar (or reduced) magnatude is seen here in our data. 
__________________________